In [28]:
from sympy import isprime
from sympy import primitive_root, factorint
from sympy import factorint

def ceil(x):
    return int(x) if x == x // 1 else int(x // 1 + 1)

def find_group(n_bits_p, q):
    r_start = ceil(2**(n_bits_p - 1) / q)
    r_end = 2**n_bits_p - 1
    i = r_start
    while(i != r_end):
        if isprime(i * q + 1):
            print(f'i = {i}')
            break
        i += 1
    r = i
    p = q * r + 1
    assert(isprime(p))
    g = pow(2, r, p)
    assert(g != 1)
    assert(pow(g, q, p) == 1)
    return p, g, r

def primitive_nth_root_mod_q(q: int, n: int):
    assert (q-1) % n == 0, "n must divide p-1"
    g = primitive_root(q)             # primitive root modulo p
    root = pow(g, (q - 1) // n, q)        # candidate of order n
    # verify primitivity
    if pow(root, n, q) != 1:
        raise RuntimeError("candidate not an n-th root")
    for r in factorint(n).keys():
        if pow(root, n // r, q) == 1:
            raise RuntimeError("not primitive (has smaller order)")

    # get inverse
    inv_root = pow(root, q - 2, q)
    assert(pow(inv_root, n, q) == 1)
    return root, inv_root

if __name__ == '__main__':
    q = 18014398509506561
    n_bits_p = 1024
    p, g, r = find_group(n_bits_p, q)

    n = 2**12
    root_nth, root_nth_inv = primitive_nth_root_mod_q(q, n)
    root_2nth, root_2nth_inv = primitive_nth_root_mod_q(q, 2 * n)
    g1 = 45166903464448924280053243999855094667742421468529127038000390690990555413931211221819477288776566894331153249407833821645949924742511460070046075201900441040131011971807761465785928717328671867197258734612868374355511121959746617532947112651893101356660662278786690036661631961050681853982159875764050554114
    r1 = 4989600773829992505939281774675235167160739812342925804737541532336365734955082555252981441562650799135269463894280377252737218446399994046048621150492088786787399460885177398506151463611780180313959299437942577336618264065630845335441386156556402970639985534021305180832638133006171157561522
    p1 = 89884656743115800376065866080041538869586058840319453933641838155019455958415370128385800572962392718049474671151114996509417426331159167274173985578890984435612970736081155406155170101580895226642186535601576839882976877204283262818534938279529430261551494225065330535924818135470675393868825060316220145843
    assert(g == g1)
    assert(r == r1)
    assert(p == p1)
    assert(p == q * r + 1)

    assert(root_nth == 5194839201355896)
    assert(root_nth_inv == 3071585657230524)
    assert(root_2nth == 9455140237568613)
    assert(root_2nth_inv == 1909215453544473)

i = 4989600773829992505939281774675235167160739812342925804737541532336365734955082555252981441562650799135269463894280377252737218446399994046048621150492088786787399460885177398506151463611780180313959299437942577336618264065630845335441386156556402970639985534021305180832638133006171157561522
